In [159]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np 
import glob
import re

In [160]:
def gsmooth(x_array, y_array, var_y, vexp = .001, nsig = 5.0):
    #returns smoothed flux array
    # if no variance array create a constant one
    if len(var_y) == 0:
        var_y = np.zeros(len(y_array))
        
    for i in range(len(var_y)):
        if var_y[i] == 0:
            var_y[i] = 1E-20
            # var_y[i] = 1E-31
    
    # Output y-array
    new_y = np.zeros(len(x_array), float)
    
    # Loop over y-array elements
    for i in range(len(x_array)):
        
        # Construct a Gaussian of sigma = vexp*x_array[i]
        gaussian = np.zeros(len(x_array), float)
        sigma = vexp*x_array[i]
        
        # Restrict range to +/- nsig sigma
        sigrange = np.nonzero(abs(x_array-x_array[i]) <= nsig*sigma)
        gaussian[sigrange] = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x_array[sigrange]-x_array[i])/sigma)**2)
        
        # Multiply Gaussian by 1 / variance
        W_lambda = gaussian / var_y
        
        # Perform a weighted sum to give smoothed y value at x_array[i]
        W0 = np.sum(W_lambda)
        W1 = np.sum(W_lambda*y_array)
        new_y[i] = W1/W0

    # Return smoothed y-array
    return new_y

In [161]:
def deredshift(wavelength, z):
    dered_wave = wavelength / (1+z)
    return dered_wave
dered_wave = deredshift(wavelength, z)

In [162]:
def find_min_wave(dered_wave, sm_flux, w1 = 5900., w2 = 6500.):
    elem_range = np.where((dered_wave > w1) & (dered_wave < w2)) #set domain as within minwave and maxwave
    list(elem_range)
    elem_wavelist = dered_wave[elem_range] 
    elem_fluxlist = sm_flux[elem_range] #find corresponding flux vales for wavelengths within domain
    elem_flux = min(elem_fluxlist) #find minimum value within these flux vales to locate "dip"
    elem_wavelength = dered_wave[np.where(sm_flux == elem_flux)][0] #find the corresponding wavelength
    return(elem_wavelength)

In [163]:
def find_velocity(min_wave, rest_wave = 6355.):
    wavelength_prop = min_wave / rest_wave
    ejecta_velocity = c * (wavelength_prop ** 2 - 1) / (wavelength_prop ** 2 + 1)
    return ejecta_velocity

In [164]:
def get_SN_name(file):
    #create a list out of the first 8 filename characters (sn0000__)
    sn_char = list(file)
    sn_namelist = [sn_char[0], sn_char[1],sn_char[2],sn_char[3],sn_char[4],sn_char[5],sn_char[6],sn_char[7],sn_char[8]]
    #in cases where SN name containes one letter, delete the "d" or "-" that will be included due to going too far in filename
    if sn_namelist[7] == '-':
        sn_namelist.remove(sn_namelist[8])
        sn_namelist.remove(sn_namelist[7])
    elif sn_namelist[8] == '-':
        sn_namelist.remove(sn_namelist[8])
    elif sn_namelist[8] == 'a':
        sn_namelist.remove(sn_namelist[8])
        sn_namelist.remove(sn_namelist[7])
    elif sn_namelist[8] == 'd':
        sn_namelist.remove(sn_namelist[8])
    
    #join list back into string that is the lowercase supernova name    
    sn_name = ''.join(sn_namelist)
    return sn_name
    print(sn_name)

In [165]:
def make_z_dict(file):
    #reads in CfA4_info.txt
    sn_datatext = open(file, "r")
    sn_datainfo = sn_datatext.readlines()
    start = False
    for line in sn_datainfo:
        if line.split()[0] == '2006ct':
                start = True
        if start:
            line_elems = line.split()
            name = line_elems[0]
            z = line_elems[10]
            z_dict[name] = float(z)
    return z_dict

In [166]:
#TEST YOUR CODE ON ONE OBJECT (plot original spectrum, deredshifted spectrum, and the smoothed deredshifted spectrum, print velocity)
#YOUR CODE GOES HERE

In [167]:
#FINAL SCRIPT STEPS
#1) Get array of all flm file names
#2) Make redshift dictionary
#3) loop through files
    #a) read in spectrum
    #b) get SN name from file name
    #c) get redshift from dictionary
    #d) deredshift
    #e) smooth
    #f) find wavelength where Si line is minimum
    #g) find and print velocity (and SN name)

In [168]:
#YOUR CODE GOES HERE
make_z_dict("CfA4_info.txt")
sndata = glob.glob("*.flm")
wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
z = 0.017192 #obtain from NED
c = 299792 #speed of light in km/s
print(sndata)
dered_wave = deredshift(wavelength, z)
sm_flux = gsmooth(dered_wave, flux, flux_error)
min_wave = find_min_wave(dered_wave, sm_flux)
print(min_wave)
v = find_velocity(min_wave)
print(v)

['sn2007fsdata.flm', 'sn2007hjdata.flm', 'sn2007fbdata.flm', 'sn2007Adata.flm', 'sn2007jgdata.flm']
6129.28532666
-10836.8784392


In [169]:
make_z_dict("CfA4_info.txt")

{'2006ct': 0.0315,
 '2006ou': 0.0135,
 '2007A': 0.0177,
 '2007aj': 0.011,
 '2007bj': 0.0167,
 '2007cb': 0.0366,
 '2007cc': 0.0291,
 '2007cf': 0.0329,
 '2007cn': 0.0253,
 '2007cs': 0.0176,
 '2007ev': 0.0427,
 '2007fb': 0.018,
 '2007fq': 0.0425,
 '2007fs': 0.0172,
 '2007hg': 0.03,
 '2007hj': 0.0141,
 '2007hu': 0.0354,
 '2007if': 0.0742,
 '2007ir': 0.0352,
 '2007is': 0.0297,
 '2007jg': 0.0371,
 '2007kd': 0.0242,
 '2007kf': 0.046,
 '2007kg': 0.007,
 '2007kh': 0.05,
 '2007kk': 0.041,
 '2007le': 0.0067,
 '2007nq': 0.045,
 '2007ob': 0.0339,
 '2007rx': 0.0301,
 '2007ss': 0.0155,
 '2007su': 0.0279,
 '2007sw': 0.0252,
 '2007ux': 0.0309,
 '2008A': 0.0165,
 '2008C': 0.0166,
 '2008Q': 0.06,
 '2008Y': 0.0697,
 '2008Z': 0.021,
 '2008ac': 0.0528,
 '2008ad': 0.05,
 '2008ae': 0.0301,
 '2008ai': 0.0353,
 '2008ar': 0.0261,
 '2008at': 0.035,
 '2008bi': 0.0134,
 '2008bw': 0.0331,
 '2008by': 0.045,
 '2008bz': 0.0603,
 '2008cd': 0.0074,
 '2008cf': 0.046,
 '2008cm': 0.0111,
 '2008dr': 0.0414,
 '2008ds': 0.021,